In [ ]:
%pip install openai langchain google-api-python-client

In [5]:
# LangChain Settings

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import AzureOpenAI

# The language model we're going to use to control the agent.
llm = AzureOpenAI(temperature=0.9, deployment_name="textdavinci003")

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [6]:
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"
)



> Entering new  chain...
 I will need to use google search to find out the temperature and then use a calculator to compute the final answer.
Action: google_search
Action Input: High temperature in SF yesterday
Observation: Weather reports from the last weeks in San Francisco with highs and lows. High: 69.8ºf @12:00 PM Low: 57.2ºf @4:10 AM Approx. Precipitation / Rain Total: in. Time (PDT), Temp. (ºf), Humidity (%), Dew Point ( ... San Francisco Weather Forecasts. ... Almanac for June 18, 2023. Forecast. Average *. Record. Temperature. High. 67 °F. 72 °F. 91 °F. Low. 54 °F. 54 °F. Sep 6, 2022 ... Bay Area heat wave: More cities break all-time high temps Tuesday ... recorded an astounding 116 degrees, which it also recorded yesterday. Sep 6, 2022 ... "Early morning temperatures in the East Bay and higher terrain are running 5 to 10 degrees higher today than they were yesterday," Brooke ... Jun 11, 2019 ... San Francisco International Airport yesterday reached its highest temperature e

'The temperature in SF yesterday raised to the .023 power is 1.1025763550530143.'